### Garbage Image Detection project using Tensor Flow 2.0

#### The model building process entails

1. creating and training of a CNN(Convolutional Neural Network) using Tensor Flow 2.0
2.Performing data processing and Augmentation 
3. Randomization
4.Dealing with image datasets

#### Prerequisite 
python programming language

linear and logistic regression(Bricks of neural networks)

Basic understanding of image processing

Basic understanding of Neural network and artificial neural networks.



### Importing libraries

In [ ]:
### Brief summary of the next steps of codes
The data would be loaded to path.
The function node takes an image path and it's labeled as arguments.

First, we load the image font as an encoded string using the rete underscore file function provided

by tenths of law.

Next, we decode this encoded string using decode underscored JPEG function.

then we return the image Tensors and the label.



First, we create our sequential object and we define transformations in it.


The resizing function is defined inside the layers API and preprocessing module.

This function takes in two arguments.

First is the height, and second is that the initial size of the image in this dataset is 48 by 48,

which is very small compared to the current deep learning standards.

But we also have to think about noise by upscaling it.

Here we are upscaling it to ninety six, which is double in terms of the initial height and weight.

Next, we have our data augmentation.

We again start a sequential object and define the augmentation methods in it.

The first augmentation is random flip.

We pass on horizontal as an argument to it, notifying it to rotate the image horizontally in a random

fashion.

Next, we have random rotate.

Now this augmentation is used to rotate an image at a certain angle.

Next, we have random zoom as well as the name suggests it is going to zoom in or zoom out of the image

randomly

If the argument is negative, the function would zoom in and if the argument is positive, it would

zoom out.

Zooming in, the image would allow the model to lower the core features in the image effectively.

So according to our arguments, we are randomly zooming in five to 10 percent.

Now that all of these augmentations are wrapped up in the sequential object, we can simply call this

object as a function and apply all these transformations in a sequence.


In [ ]:
pip install opencv-python --upgrade

In [ ]:
from __future__ import absolute_import, division, print_function
# for data visualization and data wrangling 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## For reading files to path import
import os
import pathlib
from pathlib import Path

## For Modelling 
from tqdm import tqdm
from keras.preprocessing import image
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob, os, random


tf.random.set_seed(4)

### Getting the dataset

In [ ]:
print(os.listdir("../input/split-garbage-dataset"))
#print(os.listdir('../input/split-garbage-dataset/null'))

In [ ]:
len(os.listdir("../input/split-garbage-dataset"))

In [ ]:
# Creating the Pathlib PATH objects

train_path = Path("../input/split-garbage-dataset/train")
test_path = Path("../input/split-garbage-dataset/test")
valid_path = Path("../input/split-garbage-dataset/valid")

In [ ]:
# Getting Image paths and reading 10 items

train_image_paths = list(train_path.glob("*/*"))
train_image_paths = list(map(lambda x : str(x) , train_image_paths))

train_image_paths[:10]

In [ ]:
# function for getting image respective labels using map funtion

def get_label(image_path):
    return image_path.split("/")[-2]

train_image_labels = list(map(lambda x : get_label(x) , train_image_paths))
train_image_labels[:10]

### Encoding 

In [ ]:
from sklearn.preprocessing import LabelEncoder 

Encoder = LabelEncoder()
train_image_labels = Encoder.fit_transform(train_image_labels)

train_image_labels[:10]

In [ ]:
# getting one hot encoded values of unique label

train_image_labels = tf.keras.utils.to_categorical(train_image_labels)

train_image_labels[:10]

In [ ]:
from sklearn.model_selection import train_test_split 

Train_paths , Val_paths , Train_labels , Val_labels = train_test_split(train_image_paths , train_image_labels , test_size = 0.25)

### Class Balancing

since we are training a multiclass classifier we need to take care of class inbalances.
We calculate the amount of weight that has to be given to each class. 

In [ ]:
# Compute class weights 

classTotals = Train_labels.sum(axis=0)
classWeight = classTotals.max() / classTotals

class_weight = {e : weight for e , weight in enumerate(classWeight)}
print(class_weight)

Observations:
    The weight of the classes are quiet close except for class 5 which contains plastic hence we see higher  weight getting assigned to plastic.

In [ ]:
# Function used for Transformation

def load(image , label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image , channels = 3)
    return image , label

###  Define image size  and batch size

In [ ]:
 
IMG_SIZE = 96 
BATCH_SIZE = 32

# Basic Transformation
resize = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE)          
])

In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor = (-0.1, -0.05))
])

In [ ]:
# Function used to Create a Tensorflow Data Object
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(paths , labels , train = True):
    image_paths = tf.convert_to_tensor(paths)
    labels = tf.convert_to_tensor(labels)

    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)

    dataset = tf.data.Dataset.zip((image_dataset , label_dataset))

    dataset = dataset.map(lambda image , label : load(image , label))
    dataset = dataset.map(lambda image, label: (resize(image), label) , num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(BATCH_SIZE)

    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label) , num_parallel_calls=AUTOTUNE)
    
    dataset = dataset.repeat()
    return dataset

In [ ]:
# Creating Train Dataset object and Verifying it
%time train_dataset = get_dataset(Train_paths , Train_labels)

image , label = next(iter(train_dataset))
print(image.shape)
print(label.shape)

In [ ]:
# View a sample Training Image
print(Encoder.inverse_transform(np.argmax(label , axis = 1))[0])
plt.imshow((image[0].numpy()/255).reshape(96 , 96 , 3))

In [ ]:
%time val_dataset = get_dataset(Val_paths , Val_labels , train = False)

image , label = next(iter(val_dataset))
print(image.shape)
print(label.shape)

In [ ]:
# View a sample Validation Image
print(Encoder.inverse_transform(np.argmax(label , axis = 1))[0])
plt.imshow((image[0].numpy()/255).reshape(96 , 96 , 3))

In [ ]:
# Building EfficientNet model ...Normalization is already included as part of the model

from tensorflow.keras.applications import EfficientNetB2

backbone = EfficientNetB2(
    input_shape=(96, 96, 3),
    include_top=False
)

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.summary()

In [ ]:
# Compiling the model by providing the Optimizer , Loss and Metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-06),
    loss = 'categorical_crossentropy',
    metrics=['accuracy' , tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(Train_paths)//BATCH_SIZE,
    epochs=10,
    validation_data=val_dataset,
    validation_steps = len(Val_paths)//BATCH_SIZE,
    class_weight=class_weight
)

In [ ]:
model.layers[0].trainable = False

In [ ]:
# Defining our callbacks 
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_weights.h5",verbose=1,save_best_only=True,save_weights_only = True)
early_stop = tf.keras.callbacks.EarlyStopping(patience=4)

In [ ]:
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(Train_paths)//BATCH_SIZE,
    epochs=8,
    callbacks=[checkpoint , early_stop],
    validation_data=val_dataset,
    validation_steps = len(Val_paths)//BATCH_SIZE,
    class_weight=class_weight
)

### Testing the Model

In [ ]:
from tensorflow.keras.applications import EfficientNetB2

backbone = EfficientNetB2(
    input_shape=(96, 96, 3),
    include_top=False
)

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-06),
    loss = 'categorical_crossentropy',
    metrics=['accuracy' , tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
model.load_weights("best_weights.h5")

In [ ]:
# Create a Dataset Object for 'Testing' Set just the way we did for Training and Validation
test_image_paths = list(test_path.glob("*/*"))
test_image_paths = list(map(lambda x : str(x) , test_image_paths))
test_labels = list(map(lambda x : get_label(x) , test_image_paths))

test_labels = Encoder.transform(test_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

test_image_paths = tf.convert_to_tensor(test_image_paths)
test_labels = tf.convert_to_tensor(test_labels)

def decode_image(image , label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image , channels = 3)
    image = tf.image.resize(image , [96 , 96] , method="bilinear")
    return image , label

test_dataset = (
     tf.data.Dataset
    .from_tensor_slices((test_image_paths, test_labels))
    .map(decode_image)
    .batch(BATCH_SIZE)
)

In [ ]:
# Verify Test Dataset Object
image , label = next(iter(test_dataset))
print(image.shape)
print(label.shape)

In [ ]:
# View a sample Validation Image
print(Encoder.inverse_transform(np.argmax(label , axis = 1))[0])
plt.imshow((image[0].numpy()/255).reshape(96 , 96 , 3))

### Model Evaluation

In [ ]:
# Evaluating the loaded model
loss, acc, prec, rec = model.evaluate(test_dataset)

print(" Testing Acc : " , acc)
print(" Testing Precision " , prec)
print(" Testing Recall " , rec)

Observation: The Model have have acccuracy of over 84 percent, precision of about 85 percentand 83 percent recall.

### Saving the Model 

In [ ]:
# Save Model
model.save("GarbageImageclassifierModel.h5")

In [ ]:
# Save Label Encoder 
import pickle

def save_object(obj , name):
    pickle_obj = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_obj)
    pickle_obj.close()

In [ ]:
save_object(Encoder,"LabelEncoder")

## For real time usage

### Import libraries

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

import dlib
import pickle

In [ ]:
def get_model():
    backbone = tf.keras.applications.EfficientNetB2(
        input_shape=(96, 96, 3),
        include_top=False,
        weights=None
    )
    model = tf.keras.Sequential([
        backbone,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
    ])
    return model

In [ ]:
model = get_model()
model.load_weights("best_weights.h5") # Load the saved weights 

In [ ]:
# Load LabelEncoder 
def load_object(name):
    pickle_obj = open(f"{name}.pck","rb")
    obj = pickle.load(pickle_obj)
    return obj

Le = load_object("LabelEncoder")

In [ ]:
def ProcessImage(image):
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image , [96 , 96] , method="bilinear")
    image = tf.expand_dims(image , 0)
    return image

def RealtimePrediction(image , model, encoder_):
    prediction = model.predict(image)
    prediction = np.argmax(prediction , axis = 1)
    return encoder_.inverse_transform(prediction)[0]

def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

In [ ]:
VideoCapture = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()

while True :
    
    ret , frame = VideoCapture.read() 
    
    if not ret :
        break

    gray = cv2.cvtColor( frame , cv2.COLOR_BGR2GRAY)

    rects = detector(gray , 0)

    if len(rects) >= 1 :
        for rect in rects :
            (x , y , w , h) = rect_to_bb(rect)
            img = gray[y-10 : y+h+10 , x-10 : x+w+10]
            
            if img.shape[0] == 0 or img.shape[1] == 0 :
                cv2.imshow("Frame", frame)
                
            else :
                img = cv2.cvtColor(img , cv2.COLOR_GRAY2RGB)
                img = ProcessImage(img)
                out = RealtimePrediction(img , model , Le)
                cv2.rectangle(frame, (x, y), (x+w, y+h),(0, 255, 0), 2)
                z = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, str(out), (x, z), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
                
        cv2.imshow("Frame", frame)
            
    else :
        cv2.imshow("Frame", frame)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
VideoCapture.release()
cv2.destroyAllWindows()